### Done
* spacy lemmatisation don't remove proper names

### To Do
* train on more data??
* bigram trigram quad penta common phrases
* delete [music] and [laughter]

### Discussion and Findings
* Who is most popular? christian, dion, josh. no wait. deon? dionne?
* Most laughter in patch [Music] [Laughter]
* you you you you you bad transcription
* each patch (document) should be about a few topics
* each document consists of a mixture of topics
* each topic consists of a collection of words
* minimum word length 3 or 4 for dictionary? 'they'
* alpha per document topic
* common acrosss all samples: social_convention, daily_habit (read from script), now before we get started??
* now before we get starteddion
* dion, dione, dionne, deon
* what_did_you_sew

### Method
* acquire subtitles - screenshots
* supply source code on github notsal


### How it can be used
* segments find sound_check which patch

In [192]:
import pandas as pd
import en_core_web_sm
import gensim
import gensim.corpora as corpora
import nltk
import spacy
import sklearn
import gzip
import json

from nltk.corpus import stopwords
from gensim.utils import simple_preprocess, simple_tokenize
from gensim.models import CoherenceModel
from gensim.models.phrases import Phraser
from pprint import pprint

# Plotting tools
import matplotlib.pyplot as plt
#get_ipython().run_line_magic('matplotlib', 'inline')

import warnings
warnings.filterwarnings('ignore')

nlp = spacy.load("en_core_web_sm")

In [193]:
# functions

# read in subtitles and remove YouTube's auto-generated audio descriptions
def read_and_clean(patch_number):
    path = 'C:/Users/Sally/WTP/data/'
    open_patch = open(path+f"Patch {patch_number}_ At Home with Patchwork _ YouTube Live Stream - English (auto.txt").read()
    clean_patch = open_patch.replace('\n\n',' ').replace('[Music] ','').replace('[Laughter] ','')
    return clean_patch

# turn the strings into words
def sent_to_words(your_list):
    for item in your_list:
        #yield(gensim.utils.simple_preprocess(str(item),deacc=False,min_len=3))
        yield(gensim.utils.simple_preprocess(str(item),deacc=False,min_len=0))
        

# read in subtitles and remove YouTube's auto-generated audio descriptions
def read_and_clean_alt(filename):
    path = 'C:/Users/Sally/WTP/data/'
    open_patch = open(path+f"{filename}.txt").read()
    clean_patch = open_patch.replace('\n\n',' ').replace('[Music] ','').replace('[Laughter] ','')
    return clean_patch

In [194]:
p64 = read_and_clean(64)
p65 = read_and_clean(65)
p66 = read_and_clean(66)
p67 = read_and_clean(67)
p68 = read_and_clean(68)
p69 = read_and_clean(69)

p01 = read_and_clean_alt('At Home with Patchwork')
p02 = read_and_clean_alt('At Home with Patchwork _ Sat 4th April - English (auto')

all_patches = [p01,p02,p64,p65,p66,p67,p68,p69]

for z in all_patches:
    print(len(z))

66003
69581
32038
30748
35566
43389
38486
58425


In [195]:
p01

"patch workers welcome it's great to see dotted Christian I can see organized but it's great to see that the chat coming through welcome to our second youtube live show Christian down how are you how are you going Dion very good Josh I never would have thought you'd ask a question like that can we do something more original in our lives and just asking hey how you doing we can show you than that what a boring thing that has happened to you in the last two years I had a meal last night it's a Japanese meal it had garlic in it I'm on a fog map diet and I was very very sick after it what are you supposed to do Dion considering everything has garlic in it I'm supposed I know I'm supposed to ask but every time I go to a restaurant I forget to ask even though the ramifications are catastrophic anyway welcome welcome to the show guys I would give a chat about how the show is going to run today but before we do I just want to say something to you Christian Pacifica Lee how many drinks do you h

In [173]:
all_patches[0]

"patch workers welcome it's great to see dotted Christian I can see organized but it's great to see that the chat coming through welcome to our second youtube live show Christian down how are you how are you going Dion very good Josh I never would have thought you'd ask a question like that can we do something more original in our lives and just asking hey how you doing we can show you than that what a boring thing that has happened to you in the last two years I had a meal last night it's a Japanese meal it had garlic in it I'm on a fog map diet and I was very very sick after it what are you supposed to do Dion considering everything has garlic in it I'm supposed I know I'm supposed to ask but every time I go to a restaurant I forget to ask even though the ramifications are catastrophic anyway welcome welcome to the show guys I would give a chat about how the show is going to run today but before we do I just want to say something to you Christian Pacifica Lee how many drinks do you h

In [176]:
#uses simple_preprocess
data_words = list(sent_to_words(all_patches))

data_words = all_patches

In [177]:
for p in data_words:
    print(len(p))

66003
69581
32038
30748
35566
43389
38486
58425


In [24]:
type(data_words)

list

In [ ]:
# stop_words = stopwords.words('english')
# more_sw = ['go','know','think','get','want']
# stop_words.append(more_sw)

### Pre-processing

In [178]:
# Define functions for stopwords, bigrams, trigrams and lemmatization

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def make_quadgrams(texts):
    return [quadgram_mod[trigram_mod[doc]] for doc in texts]

In [179]:
#def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'PROPN']):
def lemmatization(texts):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def lemmatisation(texts):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc])
    return texts_out

In [180]:
len(data_words)

8

In [181]:
all_patches[0]

"patch workers welcome it's great to see dotted Christian I can see organized but it's great to see that the chat coming through welcome to our second youtube live show Christian down how are you how are you going Dion very good Josh I never would have thought you'd ask a question like that can we do something more original in our lives and just asking hey how you doing we can show you than that what a boring thing that has happened to you in the last two years I had a meal last night it's a Japanese meal it had garlic in it I'm on a fog map diet and I was very very sick after it what are you supposed to do Dion considering everything has garlic in it I'm supposed I know I'm supposed to ask but every time I go to a restaurant I forget to ask even though the ramifications are catastrophic anyway welcome welcome to the show guys I would give a chat about how the show is going to run today but before we do I just want to say something to you Christian Pacifica Lee how many drinks do you h

In [185]:
data_words = [doc.split(" ") for doc in all_patches]

In [186]:
data_words

[['patch',
  'workers',
  'welcome',
  "it's",
  'great',
  'to',
  'see',
  'dotted',
  'Christian',
  'I',
  'can',
  'see',
  'organized',
  'but',
  "it's",
  'great',
  'to',
  'see',
  'that',
  'the',
  'chat',
  'coming',
  'through',
  'welcome',
  'to',
  'our',
  'second',
  'youtube',
  'live',
  'show',
  'Christian',
  'down',
  'how',
  'are',
  'you',
  'how',
  'are',
  'you',
  'going',
  'Dion',
  'very',
  'good',
  'Josh',
  'I',
  'never',
  'would',
  'have',
  'thought',
  "you'd",
  'ask',
  'a',
  'question',
  'like',
  'that',
  'can',
  'we',
  'do',
  'something',
  'more',
  'original',
  'in',
  'our',
  'lives',
  'and',
  'just',
  'asking',
  'hey',
  'how',
  'you',
  'doing',
  'we',
  'can',
  'show',
  'you',
  'than',
  'that',
  'what',
  'a',
  'boring',
  'thing',
  'that',
  'has',
  'happened',
  'to',
  'you',
  'in',
  'the',
  'last',
  'two',
  'years',
  'I',
  'had',
  'a',
  'meal',
  'last',
  'night',
  "it's",
  'a',
  'Japanese',


In [211]:
#min_count_var = 2

from gensim.models import Phrases

bigram = Phrases(data_words, min_count=20, delimiter=b' ')
trigram = Phrases(bigram[data_words], min_count=1, delimiter=b' ')
quadgram = Phrases(trigram[data_words], min_count=1, delimiter=b' ')
pentagram = Phrases(quadgram[data_words], min_count=1, delimiter=b' ')
hextagram = Phrases(pentagram[data_words], min_count=1, delimiter=b' ')

# bigram = Phrases(data_words, min_count=min_count_var, delimiter=b' ')
# trigram = Phrases(bigram[data_words], min_count=min_count_var, delimiter=b' ')
# quadgram = Phrases(trigram[data_words], min_count=min_count_var, delimiter=b' ')
# pentagram = Phrases(quadgram[data_words], min_count=min_count_var, delimiter=b' ')
# hextagram = Phrases(pentagram[data_words], min_count=min_count_var, delimiter=b' ')

for sent in data_words:
    bigrams_ = [b for b in bigram[sent] if b.count(' ') == 1]
    trigrams_ = [t for t in trigram[bigram[sent]] if t.count(' ') == 2]
    quadgrams_ = [t for t in quadgram[trigram[bigram[sent]]] if t.count(' ') == 3]
    pentagrams_ = [t for t in pentagram[quadgram[trigram[bigram[sent]]]] if t.count(' ') == 4]
    hextagrams_ = [t for t in hextagram[pentagram[quadgram[trigram[bigram[sent]]]]] if t.count(' ') == 5]

    print(bigrams_)
#    print(trigrams_)
#    print(quadgrams_)
#    print(pentagrams_)
#    print(hextagrams_)

['welcome to', 'welcome to', 'want to', 'a lot', "don't understand", 'know what', 'make sure', 'make sure', 'want to', 'make sure', 'as well', 'kind of', 'I think', 'make sure', "we'll be", "we're gonna", 'want to', 'know what', "we've got", 'should be', 'a lot', 'I think', 'know what', "we've got", "we've got", "we'll be", 'as well', "we'll be", 'welcome to', 'welcome to', 'wanted to', 'for me', 'a little', "don't know", 'and then', "don't know", 'know what', 'I think', 'and then', "you've got", "you've got", 'kind of', 'a lot', 'as well', 'kind of', 'and then', "don't want", "don't want", 'as well', "don't understand", 'no no', "don't know", 'a little', 'I think', 'I think', "don't know", 'so much', 'as well', 'should be', "you've got", 'I think', 'I think', 'for me', 'know what', 'I think', "you've got", 'I think', 'I think', 'I think', 'want to', 'kind of', 'a lot', 'a lot', 'couple of', 'and then', 'a little', 'no no', "don't know", 'I think', 'as well', 'a little', 'as well', 'an

['welcome to', 'talk about', 'talk about', 'talk about', 'I think', 'I think', "we're gonna", 'should be', "don't know", 'no no', 'all right', "don't know", 'as well', 'all right', 'I think', 'I think', 'I think', "we've got", "don't know", 'sort of', "don't know", 'kind of', 'want to', "we've got", "we'll be", 'welcome to', 'a lot', 'a little', "we'll be", 'want to', 'welcome to', 'wanted to', 'wanted to', "I'm gonna", "I've got", 'for me', 'wanted to', 'kind of', 'and then', 'want to', 'wanted to', 'I think', 'a little', 'make sure', 'kind of', 'for me', 'couple of', 'a little', 'a little', "you've got", 'for me', 'kind of', "I've got", "don't want", 'want to', "don't want", 'as well', 'a lot', 'all right', 'I think', 'couple of', 'sort of', 'I think', 'or something', 'kind of', 'and then', 'for me', 'as well', "don't know", 'kind of', 'and then', 'kind of', "you've got", "you've got", "we've got", 'and then', "we've got", "don't want", "you've got", 'kind of', 'all right', 'want to'

["we've got", 'as well', 'make sure', 'kind of', 'make sure', "I've got", 'as well', 'know what', 'no no', 'little bit', "don't know", 'couple of', 'couple of', 'all right', 'I think', 'kind of', "you've got", 'I think', 'I think', 'I think', "don't want", 'should be', 'for me', "we've got", "we'll be", 'welcome to', 'welcome to', 'a little', "we'll be", 'welcome to', 'and then', 'for me', 'for me', 'for me', "I've got", "don't know", 'thank you', 'kind of', 'a lot', 'for me', 'know what', "we've got", "don't understand", "I've got", 'really good', 'for me', "don't want", 'so much', 'and then', 'as well', "don't want", 'oh wow', 'oh wow', 'oh wow', 'oh wow', 'oh wow', 'oh wow', "I've got", "I've got", 'no no', 'and then', 'I think', 'I think', "I've got", 'I think', 'kind of', 'I think', 'I think', 'a little', 'for me', 'a little', 'hang on', 'hang on', 'I think', 'kind of', "don't know", "don't know", "you've got", 'know what', 'couple of', 'know what', 'wanted to', 'as well', "don't 

In [ ]:
# before we get started
# then you're in good company
# grab your knitting needles
# do you know what <name> thinks is really good?
# what patch did you sew?
# i've been christian
# goodbye goodbye goodbye
# game boys comedy


In [204]:
# grant hackett